In [64]:
import requests
import os
import time
import numpy as np


def get_sleep_normal(mu: float=1,
                     sigma: float=0.25) -> float:
    return np.random.normal(mu, sigma)

def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # for a period defined by a Gaussian with mu=1 and sigma=0.25.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling
          headers = {'Accept-Language': 'en-US,en;q=0.5'}
          response = self.session.get(url,timeout = self.timeout, headers=headers) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id

In [1]:
from bs4 import BeautifulSoup as bs

# conn = Connector('med_prices.csv', connector_type='selenium', path2selenium='../../geckodriver')
test_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr=569433'
# response, call_id = conn.get(test_url, 'med_priceses')

# if response.ok:
#     html = response.text
# else:
#     print('Error, no response found')

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By

import time

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options, executable_path='../../geckodriver')

In [2]:
test_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr=569433'
driver.get(test_url)
elem = driver.find_element_by_id("ctl00_TopBar_Pridudvikling")
elem.click()
date_from_elem = driver.find_element_by_id('ctl00_ctl07_ctl00_DateFrom')
print('valeu before: ' + date_from_elem.get_attribute('value'))
date_from_elem.click()
date_from_elem.clear()
print('value after clear: ' + date_from_elem.get_attribute('value'))
date_from_elem.send_keys('01-01-1998')
print('value after set: ' + date_from_elem.get_attribute('value'))
date_from_elem.send_keys(Keys.ENTER)
date_from_elem.submit()
time.sleep(20)
html = driver.find_elements_by_id('ctl00_ctl07_ctl00_PriceGrid')

valeu before: 20-05-2019
value after clear: 
value after set: 01-01-1998


In [3]:
print(driver.page_source)

<html xmlns="http://www.w3.org/1999/xhtml"><head><title>
	Produktvisning - www.medicinpriser.dk
</title><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7"><link rel="StyleSheet" href="Default.css?14-11-2017" type="text/css">
    <style type="text/css">
        .width282 { width: 282px; }
        .width283 { width: 283px; }
    </style>
    <style id="ctl00_CssRules" type="text/css">.width100 { width: 100px; }
.width75 { width: 75px; }
.width95 { width: 95px; }
.width65 { width: 65px; }
.width60 { width: 60px; }
</style><link rel="stylesheet" href="Print.css" type="text/css" media="print">
    
    <script type="text/javascript" src="Scripts/Scriptaculous/prototype.js"></script>
    <script type="text/javascript" src="Scripts/Scriptaculous/scriptaculous.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/builder.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/effects.js"></script><script 

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By

import time

# options = webdriver.ChromeOptions()
# options.add_argument('--ignore-certificate-errors')
# options.add_argument("--test-type")
# options.binary_location = "/usr/bin/chromium"

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options, executable_path='../../geckodriver')

def get_medication_price_grid_html(vnr: str, sleep_for=20):
    # Create base_url based on vnr
#     base_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr={}'.format(vnr)
    base_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr=' + vnr
    # Fetch the base_url using the firefox webdriver
    driver.get(base_url)
    # Find the price development link
    price_development = driver.find_element_by_id("ctl00_TopBar_Pridudvikling")
    # Click the price development link
    price_development.click()
    # Find the date_from input box
    date_from_input = driver.find_element_by_id('ctl00_ctl07_ctl00_DateFrom')
    # Click the date_from input field
    date_from_input.click()
    # Clear the date_from input field
    date_from_input.clear()
    # Send the '01-01-1998' key stroke to the date_from input field
    date_from_input.send_keys('01-01-1998')
    # Send key <enter> stroke to the date_from input field
    date_from_input.send_keys(Keys.ENTER)
    # Submit the content of the date_from input field
    date_from_input.submit()
    # Sleep for sleep_for seconds (default=20)
    time.sleep(sleep_for)
    # Find the PriceGrid table
    # Reload page to get the data
#     price_grid = driver.find_elements_by_id('ctl00_ctl07_ctl00_PriceGrid')
#     print(driver.page_source)
    return_str = driver.page_source
    print(driver.page_source)
    return return_str

#driver.get(test_url)
#elem = driver.find_element_by_id("ctl00_TopBar_Pridudvikling")
#elem.click()
#date_from_elem = driver.find_element_by_id('ctl00_ctl07_ctl00_DateFrom')
#print('valeu before: ' + date_from_elem.get_attribute('value'))
#date_from_elem.click()
#date_from_elem.clear()
#print('value after clear: ' + date_from_elem.get_attribute('value'))
#date_from_elem.send_keys('01-01-1998')
#print('value after set: ' + date_from_elem.get_attribute('value'))
#date_from_elem.send_keys(Keys.ENTER)
#date_from_elem.submit()
#time.sleep(20)
#html = driver.find_elements_by_id('ctl00_ctl07_ctl00_PriceGrid')

# print()

# print(pric)

In [15]:
# print(html[0].text)

# print(type(driver.page_source))

<class 'str'>


In [2]:
price_grid_html = get_medication_price_grid_html('048744')

<html xmlns="http://www.w3.org/1999/xhtml"><head><title>
	Produktvisning - www.medicinpriser.dk
</title><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7"><link rel="StyleSheet" href="Default.css?14-11-2017" type="text/css">
    <style type="text/css">
        .width282 { width: 282px; }
        .width283 { width: 283px; }
    </style>
    <style id="ctl00_CssRules" type="text/css">.width100 { width: 100px; }
.width75 { width: 75px; }
.width95 { width: 95px; }
.width65 { width: 65px; }
.width60 { width: 60px; }
</style><link rel="stylesheet" href="Print.css" type="text/css" media="print">
    
    <script type="text/javascript" src="Scripts/Scriptaculous/prototype.js"></script>
    <script type="text/javascript" src="Scripts/Scriptaculous/scriptaculous.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/builder.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/effects.js"></script><script 

In [16]:
print(price_grid_html)

<html xmlns="http://www.w3.org/1999/xhtml"><head><title>
	Produktvisning - www.medicinpriser.dk
</title><meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7"><link rel="StyleSheet" href="Default.css?14-11-2017" type="text/css">
    <style type="text/css">
        .width282 { width: 282px; }
        .width283 { width: 283px; }
    </style>
    <style id="ctl00_CssRules" type="text/css">.width100 { width: 100px; }
.width75 { width: 75px; }
.width95 { width: 95px; }
.width65 { width: 65px; }
.width60 { width: 60px; }
</style><link rel="stylesheet" href="Print.css" type="text/css" media="print">
    
    <script type="text/javascript" src="Scripts/Scriptaculous/prototype.js"></script>
    <script type="text/javascript" src="Scripts/Scriptaculous/scriptaculous.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/builder.js"></script><script type="text/javascript" src="https://medicinpriser.dk/Scripts/Scriptaculous/effects.js"></script><script 

In [27]:

panodil_vnr = '048744'

price_grid_html = get_medication_price_grid_html(panodil_vnr)

In [31]:
# print(price_grid_html[0].)

Lægemiddel/
prisperiode Tilskudspris (kr.) Panodil 048744 (kr.) Pinex 056232 (kr.) Pinemol 116637 (kr.) Paracetamol Sandoz 576810 (kr.) Pamol 598708 (kr.)
20.05.2019 0,36 0,38 0,38 0,36 0,38 0,38
03.06.2019 0,35 0,38 0,35 0,36 0,36 0,38
17.06.2019 0,34 0,38 0,35 0,35 0,34 0,38
01.07.2019 0,33 0,38 0,33 0,35 0,33 0,38
15.07.2019 0,32 0,38 0,33 0,35 0,32 0,38
29.07.2019 0,32 0,32 0,36 0,35 0,32 0,38
12.08.2019 0,31 0,38 0,36 0,35 0,31 0,38


In [3]:
import re
from bs4 import BeautifulSoup as bs
soup = bs(price_grid_html)
test = soup.findAll('table', attrs={'id': re.compile(r'.*PriceGrid.*')})

In [4]:
print(len(test))
test[0]

1


<table border="1" cellspacing="0" class="grid" id="ctl00_ctl07_ctl00_PriceGrid" rules="all" style="border-collapse:collapse;">
<tbody><tr valign="middle">
<th scope="col">Lægemiddel/<br/>prisperiode</th><th scope="col">Tilskudspris (kr.)</th><th scope="col">Panodil 048744 (kr.)</th><th scope="col">Pinex 056232 (kr.)</th><th scope="col">Pinemol 116637 (kr.)</th><th scope="col">Paracetamol Sandoz 576810 (kr.)</th><th scope="col">Pamol 598708 (kr.)</th>
</tr><tr>
<td>29.12.1997</td><td>-</td><td>0,48</td><td>-</td><td>-</td><td>-</td><td>-</td>
</tr><tr class="odd">
<td>12.01.1998</td><td>-</td><td>0,48</td><td>-</td><td>-</td><td>-</td><td>-</td>
</tr><tr>
<td>26.01.1998</td><td>-</td><td>0,49</td><td>-</td><td>-</td><td>-</td><td>-</td>
</tr><tr class="odd">
<td>09.02.1998</td><td>-</td><td>0,49</td><td>-</td><td>-</td><td>-</td><td>-</td>
</tr><tr>
<td>23.02.1998</td><td>-</td><td>0,49</td><td>-</td><td>-</td><td>-</td><td>-</td>
</tr><tr class="odd">
<td>09.03.1998</td><td>-</td><td>0

In [5]:
print(type(test[0]))


<class 'bs4.element.Tag'>


In [6]:
import pandas as pd

tes_med_prices = pd.read_html('<html><body>' + str(test[0]).replace(',', '.') + '</body></html>')

In [24]:
print(type(price_grid_html))

<class 'str'>


In [7]:
print(type(tes_med_prices))
print(len(tes_med_prices))

<class 'list'>
1


In [8]:
tes_med_prices[0]

,Lægemiddel/prisperiode,Tilskudspris (kr.),Panodil 048744 (kr.),Pinex 056232 (kr.),Pinemol 116637 (kr.),Paracetamol Sandoz 576810 (kr.),Pamol 598708 (kr.)
0,29.12.1997,-,0.48,-,-,-,-
1,12.01.1998,-,0.48,-,-,-,-
2,26.01.1998,-,0.49,-,-,-,-
3,09.02.1998,-,0.49,-,-,-,-
4,23.02.1998,-,0.49,-,-,-,-
5,09.03.1998,-,0.49,-,-,-,-
6,23.03.1998,-,0.49,-,-,-,-
7,06.04.1998,-,0.51,-,-,-,-
8,20.04.1998,-,0.51,-,-,-,-
9,04.05.1998,-,0.51,-,-,-,-
